# Goals
**[Script]** Select GEO studies from the intitial batch based on favorable study traits:
- **Sufficiently powered (n>4 per group)**
- **Control/Normal/Healthy samples**
- **Sufficient sequencing depth (check spots)**
- **Correct species, tissue/cell type, and data type**

**[Selected studies]:** GSE165322, GSE206529, [ERP104602], [GSE236566]
- **GSE206529:** We demonstrated that RPE cells carrying PRPF31 mutations revealed differentially expressed and mis-spliced genes belonging to pathways in line with the observed defective phenotypes. The rescue of RPE and photoreceptor defective phenotypes by PRPF31 gene augmentation, provide the proof of concept for future therapeutic strategies.

# Packages

In [2]:
#########################
### Standard Library ####
#########################
import os
import re
import sys
import json
import math
import warnings
import subprocess
from glob import glob
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

#####################
### Data Cleaning ###
#####################
import numpy as np
import pandas as pd
import janitor as jn
import VinlandPy as vp

###################
### Public Data ###
###################
import ffq
import GEOparse
from pysradb.sraweb import SRAweb

####################
### Session Info ###
####################
import session_info

## Options

In [ ]:
warnings.simplefilter(action="once", category=Warning)

pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 200

## Functions

# Parameters

## Inputs

In [ ]:
genome_build = "GRCh38"
release_version = "ensembl.111"
geo_id = "GSE206529"
salmon_index_path = "/mnt/disks/resources/data/references/outputs/Homo_sapiens/ensembl/111/salmon_index"

r1_suffix = "_1"  # _1.fastq.gz
r2_suffix = "_2"  # _2.fastq.gz

## Outputs

In [ ]:
input_path = vp.createDir("./inputs")
gse_path = vp.createDir(os.path.join(input_path, geo_id))

# Clean metadata
- Each study has a base set of columns to keep plus some additional columns we may want to add
- **Note:** Clean each study one at a time to ensure all desired metadata is collected

In [ ]:
gse = GEOparse.get_GEO(geo=geo_id, destdir=gse_path, include_data=False, how="full", annotate_gpl=False, silent=True)
df_geo = gse.phenotype_data  # sample_ids = gse.gsms.keys(), sra_link = gse.relations

geo_file = os.path.join(gse_path, f"{geo_id}_geo_metadata.csv")
vp.writeFile(df_geo, geo_file)

base_cols_to_keep = ["series_id","geo_accession","organism_ch1","taxid_ch1","title","source_name_ch1",
                     "description","type","molecule_ch1","instrument_model","platform_id",
                     "library_selection","library_source","library_strategy"]

vp.printDims(df_geo.reorder_columns(base_cols_to_keep), showRows=(None, 1))

In [ ]:
cols_to_add = ["characteristics_ch1.1.cell line","characteristics_ch1.2.genotype","characteristics_ch1.3.time"]
df_geo_clean = df_geo[base_cols_to_keep+cols_to_add]
df_geo_clean = df_geo_clean.move(source=cols_to_add, target="taxid_ch1", position="after", axis=1)

df_geo_clean = df_geo_clean.clean_names()
chars_to_rm = "characteristics_ch[0-9]_[0-9]_|_ch[0-9]"
df_geo_clean.columns = df_geo_clean.columns.str.replace(chars_to_rm, "", regex=True)

vp.printDims(df_geo_clean, showRows=(None, 1))

# Add metadata
- **Note:** `ffq` has access to all GEO/SRA/EMBL/DDBJ/ENCODE files

## `ffq`

In [ ]:
# Get sample metadata in nested JSON format using ffq
ffq_file = os.path.join(gse_path, f"{geo_id}_ffq_metadata.json")
cmd_ffq = f"ffq -o {ffq_file} {geo_id}"
res_ffq = vp.runCmd(cmd_ffq)

In [ ]:
# Parse JSON for relevant info using jq
cmd_jq_ids = f"cat {ffq_file} | jq -r '.{geo_id}.geo_samples | keys[]'"
geo_ids = vp.runCmd(cmd_jq_ids).stdout.split("\n")[:-1]

cmd_jq_url = f"cat {ffq_file} | jq -r '.{geo_id}.geo_samples[].samples[].experiments[].runs[].files.ftp[].url'"
geo_url = vp.runCmd(cmd_jq_url).stdout.split("\n")[:-1]

cmd_jq_md5 = f"cat {ffq_file} | jq -r '.{geo_id}.geo_samples[].samples[].experiments[].runs[].files.ftp[].md5'"
geo_md5_in = vp.runCmd(cmd_jq_md5).stdout.split("\n")[:-1]

## `pysradb`
- **Note:** The GEO sample IDs do not have a dedicated column, and can be found in weird places

In [ ]:
db = SRAweb()
sra_id = db.gse_to_srp(geo_id)["study_accession"][0]
df_sra = db.sra_metadata(sra_id, sample_attribute=True, detailed=True, expand_sample_attributes=True, output_read_lengths=True)

sra_file = os.path.join(gse_path, f"{geo_id}_sra_metadata.json")
vp.writeFile(df_sra, sra_file)

sra_base_cols_to_keep = ["study_accession", "run_accession", "library_layout", "run_total_spots", "run_total_bases"]

vp.printDims(df_sra.reorder_columns(sra_base_cols_to_keep), showRows=(None, 1))

In [ ]:
sra_cols_to_add = ["library_name"]
df_sra_clean = df_sra[sra_cols_to_add+sra_base_cols_to_keep]
df_sra_clean = df_sra_clean.clean_names().rename(columns={"library_name": "geo_accession"})
vp.printDims(df_sra_clean, showRows=(None, 1))

# Select samples to download

In [ ]:
genotype_to_keep = ["Control", "Symptomatic"]  # Remove asymptomatic samples
tissue_to_keep = ["Retinal pigmented epithelium"]  # Remove retinal organoid samples

# Create sample sheet

In [ ]:
read_ends = df_sra.library_layout.str.lower().unique()[0]

if read_ends=="single":
    r1_url = geo_url
    r2_url = None
    r1_md5_in = geo_md5_in
    r2_md5_in = None
    print("single-end reads")

if read_ends=="paired":
    url_md5_dict = dict(zip(geo_url, geo_md5_in))
    r1_url = [url for url in list(url_md5_dict.keys()) if r1_suffix in url]
    r2_url = [url for url in list(url_md5_dict.keys()) if r2_suffix in url]
    r1_md5_in = [url_md5_dict.get(url) for url in r1_url]
    r2_md5_in = [url_md5_dict.get(url) for url in r2_url]
    print("paired-end reads")

In [ ]:
# Merge dataframes
df_ffq = pd.DataFrame.from_dict({"geo_accession": geo_ids,
                                 "r1_url": r1_url,
                                 "r1_md5_in": r1_md5_in,
                                 "r2_url": r2_url,
                                 "r2_md5_in": r2_md5_in})

df_meta = df_geo_clean.merge(df_sra_clean, how="left", on="geo_accession")
df_meta = df_meta.merge(df_ffq, how="left", on="geo_accession")

# # Update values
df_meta["disease_state"] = df_meta["genotype"].map({"Control": "Healthy", "Symptomatic": "AD_retinitis_pigmentosa", "Asymptomatic": "Asymptomatic", "Isogenic": "Isogenic"})
df_meta["title"] = df_meta["title"].str[-1]
df_meta["description"] = df_meta["description"].str.split("_").str[0]
df_meta["tissue_type"] = "eye"
df_meta["cell_type"] = df_meta["cell_line"].map({"iPSCs": "iPSC-RPE"})

df_meta["sample_id"] = df_meta["run_accession"]  # geo_accession
df_meta["sample_name"] = df_meta["disease_state"] + "_" + df_meta["title"]
df_meta["group"] = df_meta["disease_state"]
df_meta["genome_build"] = genome_build
df_meta["release_version"] = release_version

# Reanme and move columns
df_meta = df_meta.rename(columns={"title": "biological_replicate"})

df_meta = df_meta.move(source=["sample_id", "sample_name", "group"], target="series_id", position="before", axis=1)
df_meta = df_meta.move(source=["study_accession", "run_accession"], target="geo_accession", position="after", axis=1)
df_meta = df_meta.move(source=["genome_build","release_version","disease_state","biological_replicate","tissue_type","cell_type"], target="taxid", position="after", axis=1)
df_meta = df_meta.move(source=["tissue_type"], target="cell_type", position="before", axis=1)

df_meta = df_meta[df_meta["genotype"].isin(genotype_to_keep)]
df_meta = df_meta[df_meta["source_name"].isin(tissue_to_keep)]

min_spots = round(df_meta.run_total_spots.astype(int).min()/1e6,1)
max_spots = round(df_meta.run_total_spots.astype(int).max()/1e6,1)
print(f"Min. Spots: {str(min_spots)}M")  # Display min. spots
print(f"Max. Spots: {str(max_spots)}M")  # Display max. spots

vp.printDims(df_meta, showRows=(None, 1))

# Download raw reads
- **Note:** `pysradb` only allows downloads from SRA, not ENA or GEO

In [ ]:
read_path = vp.createDir(os.path.join(gse_path, "raw_reads"))
read_urls = df_meta[["r1_url", "r2_url"]].stack().tolist()  # Stack removes NAs
n_threads = vp.setThreads()

with ThreadPoolExecutor(max_workers=n_threads) as executor:
    executor.map(lambda url: vp.downloadFile(url, outPath=read_path, unzip=False), read_urls)

# Download supplemental files

In [ ]:
cmd_jq_supp = f"cat {ffq_file} | jq -r '.{geo_id}.supplementary_files[].url'"
supp_urls = vp.runCmd(cmd_jq_supp).stdout.split("\n")[:-1]

supp_path = vp.createDir(os.path.join(gse_path, "supplementary_files"))
for url in supp_urls:
    vp.downloadFile(url, outPath=supp_path)

# Add more metadata 

## md5 check

In [ ]:
df_meta["r1_path"] = read_path +"/" + df_meta["r1_url"].str.split("/").str[-1]

with ThreadPoolExecutor(max_workers=n_threads) as executor:
    r1_md5_out_list = list(executor.map(lambda fq: vp.checkSums(fq), df_meta["r1_path"].tolist()))
r1_md5_out_dict = {md5.split(" ")[-1].split("/")[-1].split(r1_suffix)[0]: md5.split(" ")[0] for md5 in r1_md5_out_list}

df_meta["r1_md5_out"] = df_meta["run_accession"].map(r1_md5_out_dict)
df_meta["r1_md5_check"] = df_meta["r1_md5_out"] == df_meta["r1_md5_in"]
assert df_meta["r1_md5_check"].all(), "md5sum check FAILED. Check that fastq files were transfered correctly."

In [ ]:
df_meta["r2_path"] = read_path +"/" + df_meta["r2_url"].str.split("/").str[-1]  # NaN for SE

if df_meta["r2_path"].isnull().all():
    df_meta["r2_md5_out"] = None
    df_meta["r2_md5_check"] = None
else:
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        r2_md5_out_list = list(executor.map(lambda fq: vp.checkSums(fq), df_meta["r2_path"].tolist()))
    r2_md5_out_dict = {md5.split(" ")[-1].split("/")[-1].split(r2_suffix)[0]: md5.split(" ")[0] for md5 in r2_md5_out_list}
    
    df_meta["r2_md5_out"] = df_meta["run_accession"].map(r2_md5_out_dict)
    df_meta["r2_md5_check"] = df_meta["r2_md5_out"] == df_meta["r2_md5_in"]
    assert df_meta["r2_md5_check"].all(), "md5sum check FAILED. Check that fastq files were transfered correctly."

## Read lengths

In [ ]:
def get_read_length(fq_path):
    cmd_len = f"zcat {fq_path} | head | sed -n '2p' | wc -c"
    return float(vp.runCmd(cmd_len).stdout.strip())-1

df_meta["r1_read_length"] = df_meta["r1_path"].apply(get_read_length)

if df_meta["r2_path"].isnull().all():
    df_meta["r2_read_length"] = None
else:
    df_meta["r2_read_length"] = df_meta["r2_path"].apply(get_read_length)

## Strandedness
- Strandedness assigns reads to the correct strand, especially for overlapping genes on opposite strands

In [ ]:
strand_path = vp.createDir(os.path.join(gse_path, "salmon_strandedness"))
n_threads = vp.setThreads()

# Select first and last sample to determine strandedness
if read_ends=="single":
    strand_names2files = df_meta["r1_path"].tolist()[:1] + df_meta["r1_path"].tolist()[-1:]
    strand_names2files = {os.path.basename(f).split(".")[0]: [f] for f in strand_names2files}

if read_ends=="paired":
    l1 = df_meta[["r1_path", "r2_path"]].stack().tolist()[:2]  # First paired-end reads
    d1 = {os.path.basename(l1[0]).split(".")[0].replace(r1_suffix,""): l1}
    l2 = df_meta[["r1_path", "r2_path"]].stack().tolist()[-2:]  # Last paired-end reads
    d2 =  {os.path.basename(l2[0]).split(".")[0].replace(r1_suffix,""): l2}
    strand_names2files = {**d1, **d2}
strand_names2files

In [ ]:
for fastq_name, fastq_files in strand_names2files.items():    
    cmd_strand = (
        f"/mnt/disks/resources/software/miniconda3/envs/Py-3.12/bin/salmon quant "
        f"--index {salmon_index_path} "
        f"--libType A "  # Auto-detect library type (ex. ISR for PE and reverse stranded)
        f"--threads {n_threads} "
        f"--skipQuant "
        f"--validateMappings "
        f"-o {os.path.join(strand_path, fastq_name)} "
    )
    
    if read_ends=="single":
        cmd_strand = (
            f"{cmd_strand} "
            f"-r {fastq_files[0]}"
        )
    
    if read_ends=="paired":
        cmd_strand = (
            f"{cmd_strand} "
            f"-1 {fastq_files[0]} "
            f"-2 {fastq_files[1]}"
        )
    res_strand = vp.runCmd(cmd_strand)

## Temp file for SE data
- Nextflow cannot handle null paths, so create dummy file (SE data only)

In [ ]:
# Nextflow (SE data only): Nextflow cannot handle null paths, so create dummy file
if read_ends=="single":
    df_meta["r2_path"] = read_path +"/nextflow_dummy_file.txt"

    cmd_file = f"touch {os.path.join(read_path, 'nextflow_dummy_file.txt')}"
    vp.runCmd(cmd_file)

# QC

In [ ]:
min_cols_for_metadata = [
    "sample_id","sample_name","group","series_id","geo_accession","study_accession",
    "run_accession", "organism","taxid","genome_build","release_version","disease_state",
    "biological_replicate", "tissue_type","cell_type","type","molecule","instrument_model","platform_id",
    "library_selection","library_source","library_strategy","library_layout","run_total_spots","run_total_bases",
    "r1_url","r1_path","r1_md5_in","r1_md5_out","r1_md5_check","r1_read_length",
    "r2_url","r2_path","r2_md5_in","r2_md5_out","r2_md5_check","r2_read_length"
]

assert set(min_cols_for_metadata).issubset(df_meta.columns), \
       f"Missing required columns: {set(min_cols_for_metadata) - set(df_meta.columns)}"

# Write sample sheet for alignment

In [ ]:
df_meta = df_meta.move(source=["r1_url","r1_path","r1_md5_in","r1_md5_out","r1_md5_check","r1_read_length","r2_url","r2_path","r2_md5_in","r2_md5_out","r2_md5_check","r2_read_length"], target="run_total_bases", position="after", axis=1)

meta_file = os.path.join(input_path, f"{geo_id}_sample_sheet_for_alignment.csv")
vp.writeFile(df_meta, meta_file)

res_dos2unix = vp.runCmd(f"dos2unix {meta_file}")
res_rm_quotes = vp.runCmd(f"sed -i 's/\"//g' {meta_file}")

df_meta

# Parameters for alignment
- Add these to nextflow.config

In [ ]:
print(f"Avg. R1 read lengths: {str(df_meta["r1_read_length"].mean())} bp")
print(f"Avg. R2 read lengths: {str(df_meta["r2_read_length"].mean())} bp")

print(f"S.D. R1 read lengths: {str(np.std(df_meta["r1_read_length"]))}")
print(f"S.D. R2 read lengths: {str(np.std(df_meta["r2_read_length"]))}")

In [ ]:
strandedness_files = glob(os.path.join(strand_path, "**", "*"), recursive=True)
strandedness_files = [f for f in strandedness_files if f.endswith("lib_format_counts.json")]

for strandedness_file in strandedness_files:
    with open(strandedness_file, "r") as file:
        library_dict = json.load(file)
        print(library_dict["expected_format"])
        print(library_dict)

# Session info

In [3]:
session_info.show(os=True, std_lib=False, dependencies=False)